# importing required libraries

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout

# Reading the dataset

In [11]:
df = pd.read_csv("music.csv")

In [12]:
df

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00000.wav,0.349943,0.130225,1784.420446,2002.650192,3806.485316,0.083066,-113.596748,121.557297,-19.158825,...,8.810669,-3.667368,5.751691,-5.162763,0.750948,-1.691938,-0.409953,-2.300209,1.219929,blues
1,blues.00001.wav,0.340983,0.095918,1529.835316,2038.617579,3548.820207,0.056044,-207.556793,124.006721,8.930560,...,5.376803,-2.239120,4.216963,-6.012273,0.936110,-0.716537,0.293876,-0.287431,0.531573,blues
2,blues.00002.wav,0.363603,0.175573,1552.481958,1747.165985,3040.514948,0.076301,-90.754387,140.459900,-29.109968,...,5.789265,-8.905224,-1.083720,-9.218359,2.455806,-7.726901,-1.815723,-3.433434,-2.226821,blues
3,blues.00003.wav,0.404779,0.141191,1070.119953,1596.333948,2185.028454,0.033309,-199.431152,150.099213,5.647593,...,6.087677,-2.476421,-1.073890,-2.874778,0.780977,-3.316932,0.637982,-0.619690,-3.408233,blues
4,blues.00004.wav,0.308590,0.091563,1835.494603,1748.362448,3580.945013,0.101500,-160.266037,126.198807,-35.605450,...,-2.806384,-6.934123,-7.558618,-9.173553,-4.512165,-5.453538,-0.924161,-4.409333,-11.703781,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,rock.00095.wav,0.351991,0.079469,2008.581132,2106.617024,4254.215942,0.089267,-153.632309,109.857262,-23.085709,...,12.350816,-12.306271,5.780973,-10.279924,1.791489,-13.304210,2.473193,-6.717574,-1.189238,rock
996,rock.00096.wav,0.398653,0.076452,2006.051164,2068.327905,4147.374921,0.097659,-142.424210,116.219780,-32.177074,...,14.065345,-16.394691,4.152589,-4.350760,3.736455,-10.845638,1.875218,-7.459579,-2.802677,rock
997,rock.00097.wav,0.432103,0.081617,2077.190361,1926.989678,4030.767293,0.121824,-125.031311,115.194977,-47.993507,...,10.481961,-17.951916,1.211113,-11.534864,1.844774,-12.847901,3.447425,-12.594178,-2.107002,rock
998,rock.00098.wav,0.362349,0.083888,1398.672358,1818.148469,3014.740104,0.048731,-224.972305,123.656891,-9.754534,...,7.683081,-10.071786,-0.255098,-5.276486,-2.816289,-4.416438,1.558265,-5.043121,-3.585596,rock


In [18]:
df.isnull().sum()

chroma_stft           0
rmse                  0
spectral_centroid     0
spectral_bandwidth    0
rolloff               0
zero_crossing_rate    0
mfcc1                 0
mfcc2                 0
mfcc3                 0
mfcc4                 0
mfcc5                 0
mfcc6                 0
mfcc7                 0
mfcc8                 0
mfcc9                 0
mfcc10                0
mfcc11                0
mfcc12                0
mfcc13                0
mfcc14                0
mfcc15                0
mfcc16                0
mfcc17                0
mfcc18                0
mfcc19                0
mfcc20                0
label                 0
dtype: int64

In [19]:
df[['label']] = df[['label']].apply(lambda col:pd.Categorical(col).codes)

In [20]:
df

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,0.349943,0.130225,1784.420446,2002.650192,3806.485316,0.083066,-113.596748,121.557297,-19.158825,42.351028,...,8.810669,-3.667368,5.751691,-5.162763,0.750948,-1.691938,-0.409953,-2.300209,1.219929,0
1,0.340983,0.095918,1529.835316,2038.617579,3548.820207,0.056044,-207.556793,124.006721,8.930560,35.874683,...,5.376803,-2.239120,4.216963,-6.012273,0.936110,-0.716537,0.293876,-0.287431,0.531573,0
2,0.363603,0.175573,1552.481958,1747.165985,3040.514948,0.076301,-90.754387,140.459900,-29.109968,31.689013,...,5.789265,-8.905224,-1.083720,-9.218359,2.455806,-7.726901,-1.815723,-3.433434,-2.226821,0
3,0.404779,0.141191,1070.119953,1596.333948,2185.028454,0.033309,-199.431152,150.099213,5.647593,26.871927,...,6.087677,-2.476421,-1.073890,-2.874778,0.780977,-3.316932,0.637982,-0.619690,-3.408233,0
4,0.308590,0.091563,1835.494603,1748.362448,3580.945013,0.101500,-160.266037,126.198807,-35.605450,22.153301,...,-2.806384,-6.934123,-7.558618,-9.173553,-4.512165,-5.453538,-0.924161,-4.409333,-11.703781,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.351991,0.079469,2008.581132,2106.617024,4254.215942,0.089267,-153.632309,109.857262,-23.085709,59.040672,...,12.350816,-12.306271,5.780973,-10.279924,1.791489,-13.304210,2.473193,-6.717574,-1.189238,9
996,0.398653,0.076452,2006.051164,2068.327905,4147.374921,0.097659,-142.424210,116.219780,-32.177074,49.122971,...,14.065345,-16.394691,4.152589,-4.350760,3.736455,-10.845638,1.875218,-7.459579,-2.802677,9
997,0.432103,0.081617,2077.190361,1926.989678,4030.767293,0.121824,-125.031311,115.194977,-47.993507,52.834530,...,10.481961,-17.951916,1.211113,-11.534864,1.844774,-12.847901,3.447425,-12.594178,-2.107002,9
998,0.362349,0.083888,1398.672358,1818.148469,3014.740104,0.048731,-224.972305,123.656891,-9.754534,56.595062,...,7.683081,-10.071786,-0.255098,-5.276486,-2.816289,-4.416438,1.558265,-5.043121,-3.585596,9


# specifying x and y

In [21]:
x=df.drop(columns='label', axis=1)
y=df ['label']

In [22]:
y

0      0
1      0
2      0
3      0
4      0
      ..
995    9
996    9
997    9
998    9
999    9
Name: label, Length: 1000, dtype: int8

In [23]:
x

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0.349943,0.130225,1784.420446,2002.650192,3806.485316,0.083066,-113.596748,121.557297,-19.158825,42.351028,...,-8.324325,8.810669,-3.667368,5.751691,-5.162763,0.750948,-1.691938,-0.409953,-2.300209,1.219929
1,0.340983,0.095918,1529.835316,2038.617579,3548.820207,0.056044,-207.556793,124.006721,8.930560,35.874683,...,-5.560388,5.376803,-2.239120,4.216963,-6.012273,0.936110,-0.716537,0.293876,-0.287431,0.531573
2,0.363603,0.175573,1552.481958,1747.165985,3040.514948,0.076301,-90.754387,140.459900,-29.109968,31.689013,...,-13.123111,5.789265,-8.905224,-1.083720,-9.218359,2.455806,-7.726901,-1.815723,-3.433434,-2.226821
3,0.404779,0.141191,1070.119953,1596.333948,2185.028454,0.033309,-199.431152,150.099213,5.647593,26.871927,...,-3.196314,6.087677,-2.476421,-1.073890,-2.874778,0.780977,-3.316932,0.637982,-0.619690,-3.408233
4,0.308590,0.091563,1835.494603,1748.362448,3580.945013,0.101500,-160.266037,126.198807,-35.605450,22.153301,...,-13.083821,-2.806384,-6.934123,-7.558618,-9.173553,-4.512165,-5.453538,-0.924161,-4.409333,-11.703781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.351991,0.079469,2008.581132,2106.617024,4254.215942,0.089267,-153.632309,109.857262,-23.085709,59.040672,...,-12.380283,12.350816,-12.306271,5.780973,-10.279924,1.791489,-13.304210,2.473193,-6.717574,-1.189238
996,0.398653,0.076452,2006.051164,2068.327905,4147.374921,0.097659,-142.424210,116.219780,-32.177074,49.122971,...,-12.982494,14.065345,-16.394691,4.152589,-4.350760,3.736455,-10.845638,1.875218,-7.459579,-2.802677
997,0.432103,0.081617,2077.190361,1926.989678,4030.767293,0.121824,-125.031311,115.194977,-47.993507,52.834530,...,-16.048916,10.481961,-17.951916,1.211113,-11.534864,1.844774,-12.847901,3.447425,-12.594178,-2.107002
998,0.362349,0.083888,1398.672358,1818.148469,3014.740104,0.048731,-224.972305,123.656891,-9.754534,56.595062,...,-9.000731,7.683081,-10.071786,-0.255098,-5.276486,-2.816289,-4.416438,1.558265,-5.043121,-3.585596


# Train Test Splitting

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.23, random_state=42)

In [26]:
x_train

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
344,0.367070,0.149037,2330.262231,2437.482422,4645.071912,0.109596,-71.217743,97.731895,-18.354856,8.583561,...,-0.544312,-4.448906,-5.312886,-4.782356,-4.601093,-5.983273,-2.902000,-3.440550,-2.568890,-3.035945
994,0.389165,0.101979,2629.346944,2210.637680,5086.735908,0.146705,-90.184837,80.042290,-46.365463,56.044479,...,-17.438358,7.653449,-19.295282,1.188615,-14.481009,4.515449,-12.761936,2.995414,-6.416000,2.251966
481,0.520394,0.086490,1469.356540,1954.486612,3277.030184,0.051780,-216.042877,122.562660,0.934633,39.359921,...,-3.074017,8.878161,-4.359250,4.693549,-0.106366,0.912192,-1.921141,-1.778487,1.160832,-4.024864
575,0.293824,0.146875,2902.662054,2855.066321,6198.129187,0.126549,-78.188995,63.226551,-1.654216,21.019936,...,1.783601,-1.216035,0.942328,-2.561596,-7.766673,-5.931459,-3.014042,-2.459745,-1.023176,-5.110831
33,0.260669,0.133165,1430.582003,2075.602343,3246.730367,0.041970,-239.010910,108.669212,34.149349,30.242792,...,-28.052265,-11.258265,-16.484100,-7.636357,-12.796291,-15.352031,-13.732282,-7.400161,-10.083954,-13.285141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,0.261489,0.033232,1372.825807,1579.309887,2737.933444,0.072923,-295.702881,134.535095,-22.447182,40.741524,...,-9.373817,4.062370,1.628092,5.138569,5.442817,2.285079,-1.816668,-2.923229,0.135035,3.197436
270,0.365068,0.144322,1935.137600,1970.130683,4048.792184,0.094770,-99.297394,117.560928,-29.876978,40.838295,...,-8.745017,8.410169,-6.846654,0.434209,-5.308398,1.643461,-7.356874,0.516018,-1.461594,-5.921085
860,0.298980,0.025840,2395.744851,2300.864069,4792.462687,0.118533,-301.419312,77.221970,-13.584088,39.304447,...,-8.444426,-3.602143,-2.601354,-7.233517,-2.445522,0.758064,-8.966572,-2.654528,-5.498268,-8.405731
435,0.528840,0.191380,2825.323180,2830.629602,6190.487567,0.103299,-100.911568,64.279472,12.553191,26.489037,...,-1.483877,6.732931,3.861209,3.089634,1.749854,4.552235,-2.366770,5.429846,2.771856,3.726165


In [27]:
x_test

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
521,0.432826,0.047983,2914.946662,2420.454079,5864.864536,0.181399,-174.670853,75.904541,-12.795134,52.609890,...,-13.802083,13.011042,-7.862342,8.021646,-5.969605,9.532119,-6.002630,2.339213,-3.357959,2.510705
737,0.451668,0.240985,3236.413205,2953.596262,6698.776160,0.164825,4.821122,64.495216,-2.505614,10.421443,...,3.275973,1.753352,3.336132,2.352307,1.428844,-0.604925,-2.185691,-1.602393,-0.612146,-4.050126
740,0.292246,0.199324,1962.325584,2325.329148,3821.218560,0.095811,-76.898430,113.285751,-6.221458,19.950335,...,-3.762494,4.243446,2.292172,-2.917451,4.517389,3.417326,-0.542400,-0.557600,0.637496,-2.003500
660,0.540479,0.125770,2315.897707,2066.728767,4616.697083,0.139544,-58.192108,105.818123,-43.902336,58.437634,...,-15.682726,9.360525,-15.318879,1.530988,-5.974558,2.047317,-6.814984,4.211629,-3.402704,1.767219
411,0.443986,0.203807,2095.284362,2240.775519,4581.080632,0.094187,-74.175606,104.604111,-8.386396,49.659313,...,-14.952978,6.875351,-15.323663,4.773721,-11.092835,4.741563,-2.806385,6.797164,-5.803967,-3.778690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,0.363603,0.175573,1552.481958,1747.165985,3040.514948,0.076301,-90.754387,140.459900,-29.109968,31.689013,...,-13.123111,5.789265,-8.905224,-1.083720,-9.218359,2.455806,-7.726901,-1.815723,-3.433434,-2.226821
544,0.276576,0.086463,1004.778193,1247.541420,1810.464052,0.049664,-247.059555,168.001480,-19.712328,21.954924,...,-1.387007,5.621101,-0.294793,5.649328,3.203628,1.859547,7.012143,5.042737,2.456289,-0.896630
350,0.451152,0.159043,2375.973367,2258.379444,5005.928063,0.128782,-45.783978,101.753555,-32.084698,34.276272,...,-13.085623,16.486975,-9.916392,11.573791,-7.743574,6.135707,-7.366038,3.580805,-6.619126,2.844188
904,0.372834,0.208743,1345.850763,1841.748033,2939.265560,0.041099,-127.946869,134.443878,1.027332,54.440975,...,-6.700005,11.043714,-8.385376,2.111019,-6.338261,0.997383,-5.325869,-0.842191,-4.665996,-4.787677


In [28]:
y_train

344    3
994    9
481    4
575    5
33     0
      ..
106    1
270    2
860    8
435    4
102    1
Name: label, Length: 770, dtype: int8

In [29]:
y_test

521    5
737    7
740    7
660    6
411    4
      ..
2      0
544    5
350    3
904    9
536    5
Name: label, Length: 230, dtype: int8

# RandomForest

In [30]:
from sklearn.ensemble import RandomForestClassifier

In [31]:
clf = RandomForestClassifier(n_estimators = 100)

In [32]:
clf.fit(x_train, y_train)

RandomForestClassifier()

In [33]:
y_pred = clf.predict(x_test)

In [34]:
from sklearn import metrics 
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))

ACCURACY OF THE MODEL:  0.6260869565217392


In [35]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[14,  1,  1,  0,  0,  1,  5,  0,  0,  2],
       [ 0, 15,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 5,  1, 14,  1,  1,  3,  0,  1,  1,  1],
       [ 1,  0,  0, 13,  6,  0,  3,  1,  0,  0],
       [ 0,  0,  0,  0,  8,  0,  2,  2,  3,  1],
       [ 2,  1,  0,  0,  1, 21,  1,  0,  1,  0],
       [ 0,  0,  0,  0,  1,  0, 26,  0,  0,  1],
       [ 1,  0,  1,  0,  2,  0,  0, 10,  1,  0],
       [ 2,  0,  3,  2,  5,  1,  0,  1, 13,  0],
       [ 4,  0,  3,  1,  3,  1,  1,  2,  1, 10]], dtype=int64)

# DecisionTree

In [36]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [37]:
model = DecisionTreeClassifier(random_state=42)

In [38]:
parameters = {
    'min_samples_split': [20, 25, 30],
    'min_samples_leaf': [5, 8, 10],
    'max_leaf_nodes': [18, 19],
    'max_features': [2]
}

In [39]:
clf = GridSearchCV(model, parameters)
clf.fit(x_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_features': [2], 'max_leaf_nodes': [18, 19],
                         'min_samples_leaf': [5, 8, 10],
                         'min_samples_split': [20, 25, 30]})

In [40]:
y_pred_train = clf.predict(x_train)
print("Accuracy on training set: {:.2f}%".format(accuracy_score(y_train, y_pred_train) * 100))
y_pred_test = clf.predict(x_test)
print("Accuracy on test set: {:.2f}%".format(accuracy_score(y_test, y_pred_test) * 100))

Accuracy on training set: 45.45%
Accuracy on test set: 33.91%


# SVM

In [41]:
from sklearn import svm
clf = svm.SVC()
clf.fit(x_train, y_train)

SVC()

In [42]:
y_pred=clf.predict(x_test)

In [43]:
print("Accuracy on training set: {:.2f}%".format(accuracy_score(y_train, y_pred_train) * 100))
y_pred_test = clf.predict(x_test)
print("Accuracy on test set: {:.2f}%".format(accuracy_score(y_test, y_pred) * 100))

Accuracy on training set: 45.45%
Accuracy on test set: 16.96%
